In [1]:
import re
import requests
import pandas as pd
from collections import OrderedDict
from bs4 import BeautifulSoup
from urllib.parse import quote

#### 주제 검색어 검색결과에서 블로그 글 url 수집

In [ ]:
# 크롬 웹브라우저 실행
path = 'C:/SJL/GyeonggidoData_Course/Teamproject/Chrome/chromedriver.exe'
driver = webdriver.Chrome(path)

In [ ]:
# url 추출하기 (검색어, 기간 설정 날짜, 전체 수 하드코딩됨. 수정필요)

url_list_recent = [] 
query = "친환경 세제"
totalCount=78670

if totalCount%8:
    lastPage = int(totalCount/8) + 1
else:
    lastPage = int(totalCount/8)

for i in range(1,lastPage+1):
    url='https://section.blog.naver.com/Search/Post.naver?pageNo='+ str(i) + '&rangeType=PERIOD&orderBy=recentdate&startDate=2020-08-15&endDate=2021-08-15&keyword='+query
    driver.get(url)
    time.sleep(0.5)
 
    for j in range(1, 8): # 각 블로그 주소 저장
        try: 
            titles = driver.find_element_by_xpath('/html/body/ui-view/div/main/div/div/section/div[2]/div['+str(j)+']/div/div[1]/div[1]/a[1]')
            href = titles.get_attribute('href')
            url_list_recent.append(href)
        except:
            pass

In [ ]:
len(url_list_recent)

In [ ]:
len(set(url_list_recent))

In [ ]:
x = pd.Series(url_list_recent)
x.to_csv('naverblog_urls.csv', index=False)

#### 수집한 url을 순차적으로 조회해서 본문 scraping하기 using requests

In [2]:
#iframe 제거 후 blog.naver.com 붙이기
def delete_iframe(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml") 

    src_url = "https://blog.naver.com/" + soup.iframe["src"]
    
    return src_url

In [3]:
# 본문 스크래핑
def text_scraping(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml") 

    if soup.find("div", attrs={"class":"se-main-container"}):
        text = soup.find("div", attrs={"class":"se-main-container"}).get_text()
        text = text.replace("\n","") #공백 제거
        print("블로그")
        return text
    else:
        return "확인불가"

In [4]:
# 본문 + date, like, hashtag 스크래핑
def all_scraping(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml") 

    # date
    try:
        datetime = soup.find_all("span",{"class":'se_publishDate pcol2'})
    except:
        datetime="None"
    
    # like
    try:
        likes = soup.find_all('div',{'class':'u_likeit_list_module _reactionModule'})
        like_counts=soup.select_one("div em")
    except:
        like_counts="None"
    
    # 본문 content
    try:
        text = soup.find("div", attrs={"class":"se-main-container"}).get_text()
        text = text.replace("\n","") #공백 제거
    except:
        text = "None"
    
    # hashtag
    try:
        htags = soup.find_all('div',{'class':'wrap_tag'})
        hash_tags=""
        for h in htags:
            hash_tags+=h.text
    except:
        hash_tags="None"
    
    od = OrderedDict()
    od['URL'] = url
    od['Date']= datetime
    od['Like']=like_counts
    od['Content']= text
    od['Hashtag']=hash_tags
    
    return od

In [5]:
urls = pd.read_csv('naverblog_urls.csv')
url_list=urls['0']
url_list = list(set(url_list))
len(url_list)

3957

In [ ]:
data=[]
cnt=1
for r in url_list:
    try: 
        url = delete_iframe(r)
        posting = all_scraping(url)
        data.append(posting)
        cnt+=1
    except:
        pass

In [ ]:
dataframe = pd.DataFrame(data)

In [ ]:
dataframe.to_csv('naverblogs_crawling_requests2.csv', index=False, encoding="utf-8-sig")

#### 수집한 url을 순차적으로 조회해서 본문 scraping하기 using selenium

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

date_list=[] 
like_list=[]
content_list = []
hashtag_list=[]

In [ ]:
# url list 불러오기
import pandas as pd

u_list = pd.read_csv('./selenium_2months/naverblog_urls.csv')

In [ ]:
url_list = list(u_list['URL']) #2개월 --> 7,929개

In [ ]:
count=0

# 크롬 웹브라우저 실행
path = 'C:/SJL/GyeonggidoData_Course/Teamproject/Chrome/chromedriver.exe'
driver = webdriver.Chrome(path)

for link in url_list: # 수집한 url 만큼 반복
    driver.get(link) # 해당 url로 이동
    try:
        driver.switch_to.frame('mainFrame')
        time.sleep(0.7)

        # date
        publish_date='.blog2_container'
        dates = driver.find_elements_by_css_selector(publish_date)
        datetime=""
        for d in dates: datetime+=d.text
        date_list.append(datetime)

        # like
        u_like='.wrap_postcomment'
        likes = driver.find_elements_by_css_selector(u_like)
        like_counts=""
        for k in likes[1:2]: like_counts+=k.text
        like_list.append(like_counts)

        # content
        overlays = ".se-component.se-text.se-l-default"
        contents = driver.find_elements_by_css_selector(overlays)
        posting_contents=""
        for content in contents:
            posting_contents+=content.text
        content_list.append(posting_contents)

        # hashtag
        try:
            htags='.wrap_tag'
            tags = driver.find_elements_by_css_selector(htags)
            hash_tags=""
            for h in tags: hash_tags+=h.text
        except:
            pass
        hashtag_list.append(hash_tags)
        
        count+=1
        if count%100==0:
            print(count, 'postings scraped')
    except:
        pass

In [ ]:
#make into dataframe
import pandas as pd


dataframe = pd.DataFrame(
    data={
        'URL':url_list, 'Date':date_list, 'Like': like_list, 
        'Content':content_list, 'Hashtag':hashtag_list
    }
)

In [ ]:
dataframe.to_csv('naverblogs_crawling_selenium.csv',index=False, encoding="utf-8-sig")